In [11]:
import os
import sys

import pandas as pd
import numpy as np

from scipy.io import mmread

sys.path.append(os.path.abspath("./src"))

import utility as ut
import hessenberg as hg
from variables import *
from qr import QR


pd.set_option("display.max_columns", 10)
pd.set_option("display.width", 1000)
pd.set_option("display.precision", 12)
print("Setup compelete.")


Setup compelete.


# Hessenberg Transform

The Hessenberg transform is a similarity transformation used to reduce a complex matrix to hessenberg form. The following implementation performs the transform using *householder vectors*. The following cells check that for a given matrix $M \in \mathbb{C}^{n \times n}$,
$$M = UHU^{*}$$
where, H is the hessenberg form of $M$ and $U$ is a unitary matrix.

The unittests present in `/tests` check for the equivalence of the eigenvalues of $M$ and $H$. 

## Complex Matrices

In [12]:
a = -20
b = 50
n = 5
m = ut.complex_matrix(n, a, b, type_ = np.complex128)
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.conj().T - m, np.zeros((n, n)))}")


Original matrix:
                                   0                                 1                                 2                                 3                                 4
0 -3.6693435097640-1.5372430820320j  22.235922013353-12.666473385210j  -17.881163751703+1.384293931463j  11.139194773032+31.628818637794j  15.472832940360-17.668237507283j
1 -0.4405688483690-3.2783275313940j  -3.311294218397+41.211961123249j  44.085903947979-14.894783295535j   8.877852556721-16.343979828743j  13.298930196966+26.853885415345j
2  -6.103709700059-17.737630198703j  28.898985133043+13.903900376396j -13.955174460917+46.202793834763j -19.089581333807+45.632487636575j   34.677598292570-5.972704285567j
3  10.346102112190-18.856904065871j  15.160348264421+11.535216606151j  15.269777320326-14.222073831076j   5.618504107267-11.293466770058j -13.172233772849+23.127092468693j
4 -14.539865754130+27.243837598251j   29.988351343042+2.398561124470j  -5.224434390537+32.889937131614j  -13.837816332435+

## Real Matrices

### Random Matrices

In [13]:
a = -20
b = 50
n = 5
m = (b - a) * np.random.default_rng().random((n, n)) + a
print(f"Original matrix:\n {pd.DataFrame(m)}")
h, u = hg.hessenberg_transform(m) 
print(f"Hessenberg transformed:\n {pd.DataFrame(h)}")
print(f"Transformation matrix:\n {pd.DataFrame(u)}")
print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((n, n)))}")

Original matrix:
                  0                1                2                3                4
0  12.271191362111 -12.480520262765 -13.703492137224   5.208855239411 -14.755007338523
1   8.824730963388  26.952984099775  46.082370266301  29.943041144175  -7.951453191968
2  22.090775607578  48.229244050983  15.549670425950  27.890835742259  45.240811671689
3  40.208403658728   3.331333604717  49.013330181498  -9.575113737686   3.176638926891
4  14.528142777717  11.720556844629  45.031564922218 -10.546290720899 -16.453527860095
Hessenberg transformed:
                  0                1                2                3                4
0  12.271191362111   8.539205769309  17.237609260262 -12.425389404510  -7.996748540218
1 -48.925053917941  47.786852654230  56.871954158224   2.075067670991 -25.390744851850
2   0.000000000000  55.255667976279  20.101414702680 -16.156929165174  23.280833853652
3   0.000000000000   0.000000000000 -27.964600496590 -41.180322587979  12.942386411732


### Matrix Market

In [14]:
files = ["west0381", "blckhole"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	h, u = hg.hessenberg_transform(m) 
	print(f"Is the transformation similar: {np.allclose(u @ h @ u.T - m, np.zeros((m.shape[0], m.shape[0])))}")

Is the transformation similar: True
Is the transformation similar: True


# QR 

For a given matrix $M \in \mathbb{C}^{n\times n}$, in general, the $QR$ algorithm seeks to perform the following iteration:
* $Q_kR_k := M_k$
* $M_{k + 1} := R_kQ_k$

This algorithm can be made more stable and efficient in two ways. The first is to use $M$ is hessenberg form and the second is use to use shifts. When $H$ (hessenberg form of $M$), is used, the $QR$ decompisition of $H$ can be procedurally generated using Givens rotation matrices, $G$ (see the documentation for explanation and generation). The generation of the QR decomposition and then the subsequent formation of $RQ$ takes place as follows
* $R := G_1 G_2 \dots G_k H.$
* $Q := G_1 G_2 \dots G_k.$
* $H_{\text{new}} := R G_{k}^{*} G_{k - 1}^{*} \dots G_{1}^{*} = RQ.$

where $k \leq n - 2$.


## Wilkinson Shift

The Wilkinson shift employs stable shifts to accelerate convergence of the $QR$ hessenberg algorithm. In general the shifted algorithm looks as follows 
* $Q_kR_k := M_k - \sigma I$
* $M_{k + 1} := R_kQ_k + \sigma I$

The shift $\sigma$ is calculated as detailed in the documentation. The $QR$ decomposition and subsequent formation of $RQ$ is done using the hessenberg form of $M$ and Givens matrices as shown above.

(The implemented Wilkinson Shift implicitly deflates the matrix based on the last subdiagonal element.)

### Complex Matrices

In [25]:
a = -20
b = 50
n = 10
tol = 1e-8
m = ut.complex_matrix(n, a, b, type_ = np.complex128)
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvalues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

eig_vec = u[:, 0]
print(f"Eigenvector: {eig_vec}\n")
print(f"Eigenvector check: {m @ eig_vec / np.diag(r)[0]}")
print(f"{np.allclose(m @ eig_vec / np.diag(r)[0], eig_vec)}")


                  Eigenvalues (Numpy)
0  174.383990684857+154.414461812250j
1  84.1907537864080+14.7457440431160j
2  65.1889207574920-70.7898042621510j
3  28.2767378846600+38.4764613609030j
4  13.9978793113240-10.4539226807030j
5   0.653283914865+058.9819358696110j
6  -3.903268084808-053.8201608587670j
7 -43.9362476203140+75.8847968706930j
8 -58.3426195821320-17.4377429206240j
9 -74.4129642741100+44.0458960755510j
                 Eigenvalues (Script)
0  174.383990684857+154.414461812249j
1  84.1907537864070+14.7457440431160j
2  65.1889207574920-70.7898042621510j
3  28.2767378846600+38.4764613609030j
4  13.9978793113240-10.4539226807030j
5   0.653283914866+058.9819358696110j
6  -3.903268084808-053.8201608587680j
7 -43.9362476203140+75.8847968706930j
8 -58.3426193920350-17.4377429890310j
9 -74.4129644642100+44.0458961439580j
Comparing closeness of eigenvalues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: False
Mismatched elements:
                 

### Real Matrices

#### Random Matrices

In [16]:
a = -20
b = 50
n = 10
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvalues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0  148.467620445883+0.000000000000j
1   63.898063978090+0.000000000000j
2   43.283430828923+0.000000000000j
3   3.538794852295+12.187507775149j
4   3.538794852295-12.187507775149j
5 -0.7361121023890+0.0000000000000j
6  -6.228990636740+60.152283946588j
7  -6.228990636740-60.152283946588j
8  -47.891860409092+9.432401763208j
9  -47.891860409092-9.432401763208j
               Eigenvalues (Script)
0  148.467620445883+0.000000000000j
1   63.898063978090+0.000000000000j
2   43.283430828923+0.000000000000j
3   3.538794852295+12.187507775149j
4   3.538794852295-12.187507775149j
5 -0.7361121023890+0.0000000000000j
6  -6.228990636740+60.152283946588j
7  -6.228990636740-60.152283946588j
8  -47.891860409091+9.432401763208j
9  -47.891860409091-9.432401763208j
Comparing closeness of eigenvalues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [17]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)
tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.qr_wilkinson_shift(1e-128, 100)
eigs = np.sort(np.linalg.eig(qr_alg.H.astype(np.complex128))[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


#### Matrix Market

In [10]:
files = ["utm300"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.qr_wilkinson_shift(1e-128, 500)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		print(f"For matrix {file}")
		print(f"Number of mismatched eigenvalues: {mm.shape[0]}")
		print(f"Average absolute difference in mismatched values {mm['Difference'].mean()}")
		# with open("output_mm.txt", "w") as f:
		# 	f.write(f"{mm.to_string()}")
   


Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: False
For matrix utm300
Number of mismatched eigenvalues: 5
Average absolute difference in mismatched values (3.527668608616086e-14+0j)


## Double Shift (Inefficient)
For real matrices that have complex eigenvalues (that come in complex conjugate pairs) the shift as explained before can be collapsed into one step (explained in the documentation).

### Random Matrices

In [18]:
m = np.array([[7, 3, 4, -11, -9, -2],
     [-6, 4, -5, 7, 1, 12],
     [-1, -9, 2, 2, 9, 1],
     [-8, 0, -1, 5, 0, 8],
     [-4, 3, -5, 7, 2, 10],
     [6, 1, 4, -11, -7, -1]], dtype = np.float64)

tol = 1e-8
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-128, 200)
eigs = np.sort(np.linalg.eig(m)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

   Eigenvalues (Numpy)
0             5.0+6.0j
1             5.0-6.0j
2             4.0+0.0j
3             3.0+0.0j
4             1.0+2.0j
5             1.0-2.0j
   Eigenvalues (Script)
0              5.0+6.0j
1              5.0-6.0j
2              4.0+0.0j
3              3.0+0.0j
4              1.0+2.0j
5              1.0-2.0j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


In [19]:
a = -20
b = 50
n = 10
tol = 1e-8
m = (b - a) * np.random.default_rng().random((n, n)) + a
qr_alg = QR(m)
u, r = qr_alg.double_shift(1e-256, 500)
eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
print(f"{pd.DataFrame(eigs, columns = ['Eigenvalues (Numpy)'])}")
print(f"{pd.DataFrame(eigs_extracted, columns = ['Eigenvalues (Script)'])}")
b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
if not b:
    print(f"Mismatched elements:\n {mm}")

                Eigenvalues (Numpy)
0  177.591096351041+0.000000000000j
1  42.089351019031+30.965758119701j
2  42.089351019031-30.965758119701j
3   4.755131194994+57.273881100944j
4   4.755131194994-57.273881100944j
5  0.9373346187140+2.6483777631340j
6  0.9373346187140-2.6483777631340j
7 -13.805420767330+35.287097409231j
8 -13.805420767330-35.287097409231j
9  -45.604222247381+0.000000000000j
               Eigenvalues (Script)
0  177.591096351004+0.000000000000j
1  42.089351019031+30.965758119700j
2  42.089351019031-30.965758119700j
3   4.755131194994+57.273881100945j
4   4.755131194994-57.273881100945j
5  0.9373346187140+2.6483777631340j
6  0.9373346187140-2.6483777631340j
7 -13.805420767330+35.287097409231j
8 -13.805420767330-35.287097409231j
9  -45.604222247382+0.000000000000j
Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: True


### Matrix Market

In [19]:
files = ["gre__115"]
for file in files:
	mat = mmread(os.path.join("./test_matrices", ".".join([file, MATRIX_MARKET_FILE_EXT])))
	m = mat.toarray()
	tol = 1e-8
	qr_alg = QR(m)
	u, r = qr_alg.double_shift(1e-128, 800)
	eigs = np.sort(np.linalg.eig(qr_alg.H)[0])[::-1]
	eigs_extracted = np.sort(qr_alg.extract_eigs(r))[::-1]
	b, mm = ut.closeness(eigs_extracted, eigs, tol = tol)
	print(f"Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance {tol}: {b}")
	if not b:
		print(f"For matrix {file}")
		print(f"Number of mismatched eigenvalues: {mm.shape[0]}")
		print(f"Average absolute difference in mismatched values {mm['Difference'].mean()}")
		# with open("output_mm.txt", "w") as f:
		# 	f.write(f"{mm.to_string()}")
   

Comparing closeness of eigenvelaues from numpy linalg and approximated eigenvalues from the script with tolerance 1e-08: False
For matrix gre__115
Number of mismatched eigenvalues: 7
Average absolute difference in mismatched values (-3.114175584073564e-13+0j)


# Convergence

## Wilkinson Shift

The Wilkinson Shift (infact the shifted $QR$ algorithm in general) should converge quadratically. The alogorithm relies on the fact that the last subdiagonal element $h_{k - 1, k - 2}$ converges to 0, where $k$ is the 'active' size of the hessenberg matrix H. We expect that 
$$\frac{h^{i + 1}_{k - 1, k - 2}}{h^{i}_{k - 1, k - 2}} \approx \frac{(h^{i}_{k - 1, k - 2})^2}{h^{i}_{k - 1, k - 2}} \approx \;\text{constant}\; (< 1) $$
as $i \rightarrow \infty$. 

In [23]:
m = np.random.default_rng().random((5, 5))

tol = 1e-8
qr_alg = QR(m)

r = qr_alg.H.copy()
n = r.shape[0]
iter_ = 100
subdiagonal_elements = list()

print("Convergence for the Wilkinson shift.")
print(f"Query matrix\n {pd.DataFrame(m)}")
print(f"Above matrix is converted to hessenberg form when QR(m) is initialised.")
print(f"Hessenberg form of the matrix {pd.DataFrame(qr_alg.H)}.")
print(f"Performing {iter_} iterations.")
print(f"Starting...\n")

for i in range(iter_):
	# sigma_k = qr_alg.wilkinson_shift(r[n - 2 :, n - 2 :])		
	# Generate a scaled identity matrix for use in the shifts.
	shift_mat = qr_alg.H[n - 1, n - 1] * np.eye(n, dtype = r.dtype)
 
	r -= shift_mat
	# Perform a step of the QR 
	# hessenberg method.
	q, r = qr_alg.qr_hessenberg(r)
	r += shift_mat
	# print(f"Submatrix \n {pd.DataFrame(r[n - 2 :, n - 2 :])}\n")
	subdiagonal_elements.append([abs(r[n - 1, n - 2])])
 
for i in range(1, len(subdiagonal_elements)):
    subdiagonal_elements[i].append(subdiagonal_elements[i][0] / subdiagonal_elements[i - 1][0])
    
print(f"Ratio (as explained above): {np.mean(np.array(subdiagonal_elements[1:], dtype = np.float64), axis = 0)[1]}")
print(pd.DataFrame(subdiagonal_elements[:10]))	

Convergence for the Wilkinson shift.
Query matrix
                 0               1               2               3               4
0  0.605500645049  0.271424246316  0.435880084073  0.366568710688  0.463194467339
1  0.577270277209  0.630928787103  0.027716279206  0.877458635775  0.679894340095
2  0.411904154476  0.483223736835  0.923489832258  0.851682205122  0.924738361524
3  0.699008801759  0.539781237380  0.673609340057  0.489838334297  0.729817549227
4  0.260063839876  0.902306693028  0.058067143053  0.383065524220  0.404076052676
Above matrix is converted to hessenberg form when QR(m) is initialised.
Hessenberg form of the matrix                 0               1               2               3               4
0  0.605500645049 -0.692726925974 -0.308132511435  0.018639388359  0.193444147074
1 -1.029151354841  2.182183194122  0.454167503722 -0.381996544374 -0.582754566555
2  0.000000000000  0.783572573302  0.179620655441 -0.080726224705 -0.138233574079
3  0.000000000000  0.000000